In [ ]:
import time
import numpy as np
import matplotlib.pyplot as plt

N=666
num_bins=50

# I. Load and plot

In [ ]:
# Loading data
with open('first_kind.npy', 'rb') as f:
    zeroes_first_kind = np.load(f)
with open('second_kind.npy', 'rb') as f:
    zeroes_second_kind = np.load(f)
with open('critical_points.npy', 'rb') as f:
    critical_points = np.load(f)
with open('branch_points.npy', 'rb') as f:
    branch_points = np.load(f)

# Histogram of singular values
fig, ax = plt.subplots()
n, bins, patches = ax.hist(zeroes_first_kind, num_bins, density=True)
ax.set_xlabel('Eigenvalues')
ax.set_ylabel('Probability density')
ax.set_title(r'Histogram of singular values for N={}'.format(N))
fig.tight_layout()
plt.show()

In [ ]:
# Plotting critical points
plt.figure( figsize=(20,10) )
plt.scatter( np.real(critical_points), np.imag(critical_points), marker='*', c='g', label=f'Critical points (N-1={N-1})')
plt.scatter( np.real(zeroes_first_kind), np.imag(zeroes_first_kind), marker='*', c='r', label=f'Eigenvalues (N={N})')
plt.scatter( np.real(zeroes_second_kind), np.imag(zeroes_second_kind), marker='*', c='b', label=f'Zeroes of the second kind (N-1={N-1})')
plt.title( 'Eigenvalues and critical points')
plt.legend()
plt.show()

# Plotting branch points
## Transport colors to uniform
branch_points_color = np.abs( np.imag( critical_points ) )
branch_points_color /= np.max( branch_points_color )
uniform_sample = np.linspace( 0, 1, len(branch_points_color))
sorted_indices = np.argsort( branch_points_color )
branch_points_color = uniform_sample[ sorted_indices ]
## Plots as usual
plt.figure( figsize=(10,20) )
plt.scatter( np.real(branch_points),  np.imag(branch_points), marker='*', c=branch_points_color, label=f'Branch points (N-1={N-1})')
plt.scatter( np.real(branch_points), -np.imag(branch_points), marker='x', c=branch_points_color, label=f'(Conjugate) Branch points (N-1={N-1})')
plt.title( 'Branch points $m = M_\mu(z)$')
plt.legend()
plt.colorbar()
plt.show()

# II. Performance test when evaluating Stieljes transforms

Here we compare numpy vs sympy. Although it may be true that symbolic computations
using sympy might be slower, the question is to evaluate the overhead.

In [ ]:
def G_empirical(z):
    array = z[...,None]-zeroes_first_kind[...,:]
    return np.sum( 1/array, axis=-1)/len(zeroes_first_kind)

def G_prime_empirical(z):
    array = z[...,None]-zeroes_first_kind[...,:]
    return np.sum( -1/(array*array), axis=-1)/len(zeroes_first_kind)

def G_second_empirical(z):
    array = z[...,None]-zeroes_first_kind[...,:]
    return np.sum( 2/(array*array*array), axis=-1)/len(zeroes_first_kind)

def M_empirical(z):
    return z*G_empirical(z)-1

def M_prime_empirical(z):
    return z*G_prime_empirical(z) + G_empirical(z)

def M_second_empirical(z):
    return z*G_second_empirical(z) + 2*G_prime_empirical(z)

In [ ]:
import sympy as sp

z = sp.symbols("z")
sp_G_empirical = 0
for zero in zeroes_first_kind:
    sp_G_empirical = sp_G_empirical + 1/(z-zero)
sp_G_empirical = sp_G_empirical/len(zeroes_first_kind)

sp_M_empirical = z*sp_G_empirical - 1

In [ ]:
# Test
z0 = np.array( complex(1.0+1.0j) )
print(z0)
print( G_empirical(z0) )
print( sp_G_empirical.subs(z, z0).evalf() )

In [ ]:
# Benchmark
points_counts = [10, 25, 50, 100]

benchmark_results = {
    'numpy_vectorized': [],
    'numpy_loop': [],
    'sympy': []
}
for count in points_counts:
    print(f'''Running benchmark for {count} points...''')
    points = np.linspace(0, 10, count)+1.0j

    print( "  |- Numpy benchmark (in a single vectorized call)...")
    start = time.time()
    values = M_empirical( points )
    end = time.time()
    timing = end-start
    benchmark_results["numpy_vectorized"].append(timing)
    print("  |- Total time: ", timing)

    print( "  |- Numpy benchmark (in a loop)...")
    start = time.time()
    values_numpy = np.zeros_like( values )
    for i in range( len(values_numpy) ):
        values_numpy = M_empirical( points[i] )
    end = time.time()
    timing = end-start
    benchmark_results["numpy_loop"].append(timing)
    print("  |- Total time: ", timing)

    print( "  |- Sympy benchmark...")
    start = time.time()
    values_sympy = np.zeros_like( values )
    for i in range( len(values_sympy) ):
        values_sympy = sp_M_empirical.subs('z', points[i] ).evalf()
    end = time.time()
    timing = end-start
    benchmark_results["sympy"].append(timing)
    print("  |- Total time: ", timing)

    print("")
# End for

# GOSH! Huge overhead!